In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
import pickle

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Thesis/posts1352_standard_representations.csv')

In [24]:
from ast import literal_eval

token_list = []
labels = []

# for x in df['Topic_Probabilities']:
# for x in df['Topic']:
# for x in df['Representative_Docs']:
# for x in df['PreprocessedToken']:
for x in df['Topic']:
    token_list.append(x)

corpus = [token.split() for token in token_list]
#bertopic_words = bertopic_words[:1600]

for y in df['HateSpeech']:
    labels.append(y)

#y_list = y_list[:1600]

In [25]:
# obtain token list that's actually list instead of string
from ast import literal_eval

topic_words = []

for x in token_list:
    topic_words.append((literal_eval(x)))

#sequences = topic_words #only for topic probabilities

In [26]:
# change the list containing a list of elements into list of strings (not for topic probabilities)
topic_strings = []

for i in topic_words:

  temp = ' '.join(i)
  topic_strings.append(temp)

In [27]:
# Initialize the tokenizer (not for topic probabilities)
tokenizer = Tokenizer(num_words=1620)
tokenizer.fit_on_texts(topic_strings)
sequences = tokenizer.texts_to_sequences(topic_strings)

In [ ]:
# Save the tokenizer to a file
with open('/content/drive/MyDrive/Thesis/posts1352_standard_GRU_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the tokenizer from the file
with open('/content/drive/MyDrive/Thesis/posts1352_standard_GRU_tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [28]:
# Pad the sequences to have the same length
max_sequence_length = 46
data = pad_sequences(sequences, maxlen=max_sequence_length)
data = np.array(data)
labels = np.array(labels)

In [29]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2)

In [30]:
model = Sequential([
    Embedding(input_dim=1620, output_dim=128, input_length=max_sequence_length),
    GRU(units=128, return_sequences=True),
    GRU(units=128),
    Dropout(0.5),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

In [31]:
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 46, 128)           207360    
                                                                 
 gru_4 (GRU)                 (None, 46, 128)           99072     
                                                                 
 gru_5 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [32]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
41/41 [==============================] - 12s 145ms/step - loss: 0.6929 - accuracy: 0.5278 - val_loss: 0.6786 - val_accuracy: 0.5802
Epoch 2/10
41/41 [==============================] - 8s 188ms/step - loss: 0.6875 - accuracy: 0.5409 - val_loss: 0.6707 - val_accuracy: 0.5833
Epoch 3/10
41/41 [==============================] - 5s 123ms/step - loss: 0.6856 - accuracy: 0.5409 - val_loss: 0.6738 - val_accuracy: 0.5648
Epoch 4/10
41/41 [==============================] - 5s 125ms/step - loss: 0.6799 - accuracy: 0.5718 - val_loss: 0.6689 - val_accuracy: 0.5864
Epoch 5/10
41/41 [==============================] - 7s 179ms/step - loss: 0.6819 - accuracy: 0.5640 - val_loss: 0.6617 - val_accuracy: 0.5833
Epoch 6/10
41/41 [==============================] - 5s 124ms/step - loss: 0.6782 - accuracy: 0.5525 - val_loss: 0.6640 - val_accuracy: 0.5710
Epoch 7/10
41/41 [==============================] - 7s 173ms/step - loss: 0.6745 - accuracy: 0.5656 - val_loss: 0.6646 - val_accuracy: 0.5802
Epoch

In [33]:
# Evaluate the model on the validation data of Topic
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 29ms/step - loss: 0.6638 - accuracy: 0.5864
Validation Loss: 0.6638393402099609
Validation Accuracy: 0.5864197611808777


In [12]:
# Evaluate the model on the validation data of Topic Probability
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 1s 55ms/step - loss: 0.6755 - accuracy: 0.5401
Validation Loss: 0.6754544973373413
Validation Accuracy: 0.540123462677002


In [23]:
# Evaluate the model on the validation data of Representative Document
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 28ms/step - loss: 0.6859 - accuracy: 0.5247
Validation Loss: 0.6858827471733093
Validation Accuracy: 0.5246913433074951


In [ ]:
# Evaluate the model on the validation data of WordRepresentation
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

11/11 [==============================] - 0s 38ms/step - loss: 1.0015 - accuracy: 0.8519
Validation Loss: 1.0015413761138916
Validation Accuracy: 0.8518518805503845


In [ ]:
model.save('/content/drive/MyDrive/Thesis/posts1352_standard_GRU_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/posts1352_standard_GRU_model.h5')

In [1]:
# %load preprocess.py
import string
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


import spacy

nlp = spacy.load('en_core_web_sm')

stopword_extra = ["always","want","even","still","ever","also","already","yet","basically","actually","need","please","ago","probable","probably","however","instead","quite","nt","na","u","gon","lol","im","ca","us","cnt","wo","em","etc","ll","aint","r","cant","shouldnt","wont","lah","dont","never"]

stopwords = nltk.corpus.stopwords.words('english')
stopwords += stopword_extra

negative_list = ['not','never','ain','aint','no','neither','nor','nt','cant','dont',"cnt",'wont',"shouldnt"]

def preprocess(input):
    preprocessed_input = []

    input = sent_tokenize(input)

    for text in input:

        #1. Generating the list of words in the tweet (hastags and other punctuations removed)
        text_blob = TextBlob(text)
        text = ' '.join(text_blob.words)

        # remove number
        text = re.sub(r'[0-9]', '', text)

        # lowercase
        text = text.lower()

        text = text.replace('/',' ')

        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')

        text = word_tokenize(text)

        #keep tokens that are alphabet characters
        text = [t for t in text if t.isalpha()]

        # replace the negation token
        replacer  = AntonymReplacer()
        text = replacer.replace_negations(text)

        # remove the stopwords
        text = [i for i in text if i not in stopwords]

        #remove empty token
        text = [t for t in text if len(t) > 0]

        preprocessed_input.append(text)


    return preprocessed_input


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent))
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) >= 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word in negative_list and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

ModuleNotFoundError: No module named 'nltk'

In [3]:
# %load prediction.py
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import keras

max_sequence_length = 46

def change_to_string(input):
    # obtain token list that's actually list instead of string
    from ast import literal_eval

    topic_words = input

    #for x in input:
    #    topic_words.append((literal_eval(x)))

    # change the list containing a list of elements into list of strings
    topic_strings = []

    for i in topic_words:
        temp = ' '.join(i)
        topic_strings.append(temp)

    topic_string = ' '.join(topic_strings)

    topic_string = [topic_string]

    return topic_string

def tokenization(input):
    
    # Load the tokenizer from the file
    with open('posts1352_standard_GRU_tokenizer_2025_2.pickle', 'rb') as tokenizer_path:
        loaded_tokenizer = pickle.load(tokenizer_path)

    sequences = loaded_tokenizer.texts_to_sequences(input)

    padded_data = pad_sequences(sequences, maxlen=max_sequence_length)

    return padded_data

def prediction(input):

    model = tf.keras.models.load_model('posts1352_standard_GRU_model_2025_2.keras')
    #model = tf.keras.models.load_model('posts1352_standard_GRU_model.h5',compile=False)
    #model.compile() 

    input = change_to_string(input)

    input = tokenization(input)

    predictions = model.predict(input)

    binary_predictions = (predictions > 0.5).astype(int)

    return binary_predictions[0][0]

In [1]:
# %load UI.py
import tkinter as tk
from preprocess import preprocess
from prediction import prediction
import tensorflow as tf
import keras

# Function to display the input string
def show_output():
    input_string = entry.get()
    preprocessed_input_string = preprocess(input_string)
    hate_speech_value = prediction(preprocessed_input_string)
    if (hate_speech_value==1):
        result = 'hate speech'
    else:
        result = 'not hate speech'
    output_label.config(text=f"This message is {result}.")

# Create the main window
root = tk.Tk()
root.title("Message Detection System")

# Create and place the input label
label = tk.Label(root, text="Submit any message:")
label.pack(pady=10)

# Create and place the entry widget
entry = tk.Entry(root, width=50)
entry.pack(pady=10)

# Create and place the submit button
button = tk.Button(root, text="Submit", command=show_output)
button.pack(pady=10)

# Create and place the output label
output_label = tk.Label(root, text="")
output_label.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()
